In [ ]:
import csv, math
rows = []
with open('data.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        mmlu = float(row['MMLU'].strip('%'))
        bparams = float(row['B Params'])
        name = row['Model']
        rows.append((mmlu, bparams, name))

x_min, x_max = 0, 100
b_min = min(b for _, b, _ in rows if b > 0)
b_max = max(b for _, b, _ in rows)
log_y_min = math.log10(b_min)
log_y_max = math.log10(b_max)
width, height = 800, 600
margin = 60
svg = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}">']
svg.append(f'<line x1="{margin}" y1="{height-margin}" x2="{width-margin}" y2="{height-margin}" stroke="black"/>')
svg.append(f'<line x1="{margin}" y1="{height-margin}" x2="{margin}" y2="{margin}" stroke="black"/>')
for i in range(0, 101, 20):
    x = margin + (i - x_min)/(x_max - x_min)*(width-2*margin)
    svg.append(f'<line x1="{x}" y1="{height-margin}" x2="{x}" y2="{height-margin-5}" stroke="black"/>')
    svg.append(f'<text x="{x}" y="{height-margin+15}" font-size="10" text-anchor="middle">{i}%</text>')
val = 1
while val <= b_max:
    if val >= b_min:
        log_val = math.log10(val)
        y = height - margin - (log_val - log_y_min)/(log_y_max - log_y_min)*(height-2*margin)
        svg.append(f'<line x1="{margin}" y1="{y}" x2="{margin+5}" y2="{y}" stroke="black"/>')
        svg.append(f'<text x="{margin-5}" y="{y+4}" font-size="10" text-anchor="end">{val}</text>')
    val *= 10
def overlaps(box, boxes):
    for bx1, by1, bx2, by2 in boxes:
        if not (box[2] < bx1 or box[0] > bx2 or box[3] < by1 or box[1] > by2):
            return True
    return False
boxes = []
for mmlu, b, name in rows:
    x = margin + (mmlu - x_min)/(x_max - x_min)*(width-2*margin)
    log_y = math.log10(b)
    y = height - margin - (log_y - log_y_min)/(log_y_max - log_y_min)*(height-2*margin)
    label_x = x + 8
    label_y = y - 8
    text_w = len(name) * 6
    text_h = 10
    if label_y - text_h < margin:
        label_y = y + text_h + 8
    box = [label_x, label_y - text_h, label_x + text_w, label_y]
    while overlaps(box, boxes):
        label_y += text_h + 2
        box = [label_x, label_y - text_h, label_x + text_w, label_y]
    boxes.append(box)
    svg.append(f'<circle cx="{x}" cy="{y}" r="4" fill="blue"/>')
    line_y = label_y - text_h/2
    svg.append(f'<line x1="{x}" y1="{y}" x2="{label_x}" y2="{line_y}" stroke="gray"/>')
    svg.append(f'<text x="{label_x}" y="{label_y}" font-size="10">{name}</text>')
svg.append('</svg>')
with open('graph.svg', 'w') as f:
    f.write('\n'.join(svg))
print('graph.svg generated')
